# PancCancerTMA - Register images

Anna found in her master thesis that TMAs 1 and 2 of the experiment showed low transcript counts, while in case of TMA4 there were problems during sectioning leading to cores being misplaced. Therefore we will focus here on the analysis of TMA3.

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import read_xenium
from parse import parse
from insitupy import register_images

## Load Xenium data into `InSituData` object

Now the Xenium data can be parsed by providing the data path to `InSituData`

In [3]:
# data_dir = Path(r"C:\Users\ge37voy\OneDrive - TUM\data\2315_VascularCancer_Mogler\2315-02-glioblastoma\20240112__122324__2315-02-Glioblastoma\output_reseg-0017313__N-245-23-N__20240213__135522")
# img_to_be_registered = Path(r"C:\Users\ge37voy\OneDrive - TUM\data\2315_VascularCancer_Mogler\2315-02-glioblastoma\export_tiles_qpath\unregistered_images\0017313__N-245-23-N__HE__histo.ome.tif")

data_dir = Path(r"C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\output-XETG00000__0001879__Replicate 1")
if_to_be_registered = Path(r"C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\unregistered_images\0001879__Replicate 1__CD20_HER2_DAPI__IF.tif")
he_to_be_registered = Path(r"C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\unregistered_images\0001879__Replicate 1__HE__histo.tif")

In [4]:
# read Xenium data
xd = read_xenium(data_dir, metadata_filename="experiment.xenium")

## Registration of IF images

In [5]:
# parse name of current image
pattern_img_file: str = "{slide_id}__{sample_id}__{image_names}__{image_type}"
image_name_sep = "_"

# make sure to remove also suffices like .ome.tif
img_stem = if_to_be_registered.stem.split(".")[0]

# retrieve the image patterns
img_file_parsed = parse(pattern_img_file, img_stem)
channel_names = img_file_parsed.named["image_names"].split(image_name_sep)
image_type = img_file_parsed.named["image_type"] # check which image type it has (`histo` or `IF`)

In [6]:
channel_names

['CD20', 'HER2', 'DAPI']

In [7]:
register_images(
    data=xd,
    image_to_be_registered=if_to_be_registered,
    image_type=image_type,
    channel_names=channel_names,
    channel_name_for_registration="DAPI",
    template_image_name="nuclei",
    save_results=True
    )

	Processing following IF images: CD20, HER2, DAPI
		Loading images to be registered...
Loading images...
		Select image with nuclei from IF image (channel: 2)
		Load image into memory...
		Rescale image and template to save memory.
			Rescaled to following dimensions: (3314, 4827)
			Rescaled to following dimensions: (3412, 4688)
		Convert scaled images to 8 bit
		Extract common features from image and template
		2024-05-28 13:52:33: Get features...
			Method: SIFT...
		2024-05-28 13:52:39: Compute matches...
		2024-05-28 13:52:43: Filter matches...
			Sufficient number of good matches found (10308).
		2024-05-28 13:52:43: Display matches...
		2024-05-28 13:52:45: Fetch keypoints...
		2024-05-28 13:52:45: Estimate 2D affine transformation matrix...
		2024-05-28 13:52:45: Register image by affine transformation...
		Save OME-TIFF to C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\registered_images\0001879__Replicate 1__CD20__registered.ome.tif
		Save QC files to C:\Users\ge37voy

## Registration of HE image

In [11]:
# parse name of current image
pattern_img_file: str = "{slide_id}__{sample_id}__{image_names}__{image_type}"
image_name_sep = "_"

img_stem = he_to_be_registered.stem.split(".")[0] # make sure to remove also suffices like .ome.tif
img_file_parsed = parse(pattern_img_file, img_stem)
channel_names = img_file_parsed.named["image_names"].split(image_name_sep)
image_type = img_file_parsed.named["image_type"] # check which image type it has (`histo` or `IF`)

In [15]:
register_images(
    data=xd,
    image_to_be_registered=he_to_be_registered,
    image_type=image_type,
    channel_names=channel_names,
    channel_name_for_registration=None,
    template_image_name="nuclei",
    save_results=True
    )

	Processing following histo images: HE
		Loading images to be registered...
Loading images...
		Run color deconvolution
		Rescale image and template to save memory.
			Rescaled to following dimensions: (3448, 4638)
			Rescaled to following dimensions: (3412, 4688)
		Convert scaled images to 8 bit
		Extract common features from image and template
		2024-05-28 13:59:19: Get features...
			Method: SIFT...
		2024-05-28 13:59:24: Compute matches...
		2024-05-28 13:59:26: Filter matches...
			Number of good matches (2) below threshold (20). Flipping is tested.
		Vertical flip is tested.
			Method: SIFT...
		2024-05-28 13:59:30: Compute matches...
		2024-05-28 13:59:32: Filter matches...
			Sufficient number of good matches found (30).
		2024-05-28 13:59:32: Display matches...
		2024-05-28 13:59:32: Fetch keypoints...
		2024-05-28 13:59:32: Estimate 2D affine transformation matrix...
		Image is flipped vertically
		2024-05-28 13:59:32: Register image by affine transformation...
		Save OME-TIF

In [18]:
xd.load_images()

Loading images...


In [30]:
xd.show()

In [28]:
hasattr(xd.images, "HE")

True

In [29]:
xd.images

images
nuclei:	(25778, 35416)
CD20:	(25778, 35416)
HER2:	(25778, 35416)
DAPI:	(25778, 35416)
HE:	(25778, 35416, 3)

In [26]:
xd.images.HE

[dask.array<from-zarr, shape=(25778, 35416, 3), dtype=uint8, chunksize=(1024, 1024, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(12889, 17708, 3), dtype=uint8, chunksize=(1024, 1024, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(6444, 8854, 3), dtype=uint8, chunksize=(1024, 1024, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(3222, 4427, 3), dtype=uint8, chunksize=(1024, 1024, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(1611, 2213, 3), dtype=uint8, chunksize=(1024, 1024, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(805, 1106, 3), dtype=uint8, chunksize=(805, 1024, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(402, 553, 3), dtype=uint8, chunksize=(402, 553, 3), chunktype=numpy.ndarray>,
 dask.array<from-zarr, shape=(201, 276, 3), dtype=uint8, chunksize=(201, 276, 3), chunktype=numpy.ndarray>]

In [27]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset
Data folder:	output-XETG00000__0001879__Replicate 1
Metadata file:	experiment.xenium
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       DAPI:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               nuclear
               cellular
    ➤ transcripts
       DataFrame with shape 42638083 x 7

In [19]:
xd

[autoreload of insitupy._core.insitudata failed: Traceback (most recent call last):
  File "c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
  File "c:\Users\ge37voy\AppData\Local\miniconda3\envs\insitupy\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "C:\Users\ge37voy\Github\InSituPy\insitupy\_core\in

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset
Data folder:	output-XETG00000__0001879__Replicate 1
Metadata file:	experiment.xenium
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       DAPI:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               nuclear
               cellular
    ➤ transcripts
       DataFrame with shape 42638083 x 7

In [21]:
xd.images.metadata

{'nuclei': {'filename': 'morphology_mip.ome.tif',
  'shape': (25778, 35416),
  'axes': 'YX',
  'OME': {'xmlns': 'http://www.openmicroscopy.org/Schemas/OME/2016-06',
   'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
   'Creator': 'tifffile.py 2021.4.8',
   'UUID': 'urn:uuid:a73aa686-7468-11ed-89c9-06acdb970e37',
   'xsi:schemaLocation': 'http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd',
   'Plate': {'ID': 'Plate:1',
    'WellOriginX': '-0.0',
    'WellOriginXUnit': 'µm',
    'WellOriginY': '-0.0',
    'WellOriginYUnit': 'µm'},
   'Instrument': {'ID': 'Instrument:1',
    'Microscope': {'Manufacturer': '10x Genomics', 'Model': 'Xenium'}},
   'Image': {'ID': 'Image:0',
    'Name': 'Image0',
    'InstrumentRef': {'ID': 'Instrument:1'},
    'Pixels': {'DimensionOrder': 'XYZCT',
     'ID': 'Pixels:0',
     'SizeC': '1',
     'SizeT': '1',
     'SizeX': '35416',
     'SizeY': '25778',
     'SizeZ': '1',
     'Type': 'uint16